# Laboratorio de Esteganografía con OpenCV

**Curso:** CY-203 Hackeo Ético  
**Tema:** Ocultamiento de imágenes usando LSB (Least Significant Bit)  
**Objetivo:** Ocultar una imagen dentro de otra utilizando manipulación de bits con OpenCV.


## ¿Cómo funciona el ocultamiento usando 4 bits? (LSB Esteganografía)

En una imagen digital en color (formato RGB), cada color se representa con **8 bits** por canal, es decir, un valor entre 0 y 255. Por ejemplo:

```
Pixel RGB = [R, G, B] = [11001010, 10100110, 10011011]  (en binario)
```

### Estrategia

1. **Tomamos los 4 bits más significativos** de cada color de la imagen secreta.
2. **Los insertamos en los 4 bits menos significativos** de la imagen portadora.

Esto produce un cambio sutil en la imagen visible, pero guarda los datos ocultos.

### Ejemplo paso a paso

Supongamos que tenemos:

- **Canal azul** de la imagen portadora:
  ```
  cover_pixel_blue = 11001010
  ```

- **Canal azul** de la imagen secreta:
  ```
  secret_pixel_blue = 10110101
  ```

#### Paso 1: Limpiar los 4 bits menos importantes de la portada
```python
cover_clean = cover_pixel_blue & 0b11110000  # 11000000
```

#### Paso 2: Tomar los 4 bits más importantes de la secreta
```python
secret_bits = secret_pixel_blue >> 4         # 00001011
```

#### Paso 3: Combinar ambos valores
```python
stego_pixel = cover_clean | secret_bits      # 11001011
```

Este valor es el nuevo píxel camuflado.

---

### ¿Por qué usar solo 4 bits?

- **Invisibilidad:** Cambiar 4 bits tiene un impacto visual casi imperceptible.
- **Capacidad razonable:** Se pueden ocultar 12 bits por píxel (4 por canal RGB).
- **Compatibilidad:** Funciona bien incluso en imágenes comprimidas sin pérdida (como PNG).

---

### ¿Cómo se recupera la imagen secreta?

1. Extraemos los **4 bits menos significativos** del píxel camuflado:
   ```python
   hidden_bits = stego_pixel & 0b00001111
   ```

2. Los convertimos de vuelta a los bits más significativos:
   ```python
   recovered = hidden_bits << 4
   ```

Esto da una versión degradada pero visible de la imagen secreta.

---

### Visualización del proceso

| Etapa               | Valor binario del canal azul |
|---------------------|-------------------------------|
| Imagen Portadora    | `11001010`                    |
| Imagen Secreta      | `10110101`                    |
| Imagen Camuflada    | `11001011`                    |
| Bits Extraídos      | `00001011` << 4 = `10110000`  |


In [ ]:
# Requisitos
%pip install opencv-python matplotlib numpy

## 1. Importar librerías y cargar imágenes

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def show(img, title="Imagen"):
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()

# Cargar imagen portada y mensaje
cover = cv2.imread('cover.png')
secret = cv2.imread('secret.png')

# Redimensionar imagen secreta al tamaño de la portada
secret = cv2.resize(secret, (cover.shape[1], cover.shape[0]))

show(cover, "Imagen Portadora")
show(secret, "Imagen Secreta a Ocultar")

## 2. Preparar las imágenes para ocultamiento
Usamos los 4 bits más significativos de la imagen portadora y los 4 más significativos del mensaje secreto.

In [ ]:
# Convertimos los valores a 8 bits (0-255)
cover_bits = (cover >> 4) << 4         # Limpiamos los 4 bits menos significativos
secret_bits = secret >> 4              # Tomamos los 4 bits más significativos

# Ocultamos la imagen
stego = cover_bits | secret_bits

show(stego, "Imagen con Mensaje Oculto")
cv2.imwrite("stego_result.png", stego)

## 3. Extraer el mensaje oculto desde la imagen esteganográfica

In [ ]:
# Extraemos los 4 bits menos significativos (donde está el mensaje)
extracted = (stego & 0b00001111) << 4  # Restauramos a 8 bits
show(extracted, "Imagen Oculta Recuperada")
cv2.imwrite("recovered_secret.png", extracted)

## 4. Conclusión
Este laboratorio demuestra cómo la manipulación directa de bits permite ocultar información visual en otra imagen sin alterar notablemente la portada.
- Esta técnica es **vulnerable** a compresión y filtrado.
- No requiere clave secreta: es una forma de **esteganografía pura**.
- Puede extenderse a múltiples capas (8 bits, canales individuales, etc.).

**Aplicación real:** Espías, periodistas, activistas o malware podrían usar este método para ocultar datos en imágenes aparentemente inocuas.